importation des librairies necécaires

In [5]:
import pickle
import re
import spacy

# *Recuperation du dictionnaire,liste des pages et le pagerank*

la liste des pages

In [6]:
pages_list_file_path ="pages_list"


with open(pages_list_file_path , 'rb') as pickle_file:
    pages_list = pickle.load(pickle_file)

In [7]:
#change ids to int
def convert_page_ids_to_int(pages_list):
    for i, page_info in enumerate(pages_list):
        title, page_id_str, index_in_corpus = page_info
        pages_list[i] = (title, int(page_id_str), index_in_corpus)


convert_page_ids_to_int(pages_list)

le dictionnaire contenant relation mot-pages

In [8]:
#dict mot-pages 20k mots avec normalisation
dict_path = 'mot_pages'

with open(dict_path, 'rb') as pickle_file:
    d=pickle.load(pickle_file)

In [9]:
d["france"][0]

1.6292724392809057

Le page rank

In [10]:
#page rank
pages_rank_path = 'pages_rank'

with open(pages_rank_path, 'rb') as pickle_file:
    pages_rank=pickle.load(pickle_file)

In [11]:
pages_rank

{1095: 2.184996164901414e-05,
 13706: 1.4847200038705126e-05,
 52554: 1.4673430556540514e-05,
 8592685: 1.4063009555090457e-05,
 802627: 1.4040731416351406e-05,
 29600: 1.377042333298423e-05,
 9091: 1.3605565106315237e-05,
 1942: 1.3581801758326912e-05,
 2719: 1.3514967342109753e-05,
 681356: 1.3452588553640404e-05,
 3844346: 1.3399121020666678e-05,
 2996: 1.3249114886490388e-05,
 3443: 1.3149605866789283e-05,
 792151: 1.313920940204439e-05,
 776823: 1.3032274336096937e-05,
 2252009: 1.2925339270149483e-05,
 25392: 1.291642801465386e-05,
 36708: 1.2895635085164079e-05,
 52440: 1.2879297783422107e-05,
 1249: 1.2855534435433782e-05,
 2009: 1.2815433785703487e-05,
 675918: 1.2809492948706406e-05,
 63: 1.276939229897611e-05,
 3156681: 1.2650575559034493e-05,
 1051442: 1.2647605140535954e-05,
 3252248: 1.2610474909304198e-05,
 3973375: 1.2499084215608933e-05,
 10841590: 1.2491658169362582e-05,
 2894: 1.247829128611915e-05,
 36709: 1.2473835658371339e-05,
 7622: 1.2457498356629367e-05,
 7445

# *Traitement de la requete*

fonctions utiles pour le traitement de la requete (repris du TD2)

In [12]:
#extrait la liste des mots à partir du text
def text_to_words(page_text):
    return re.findall(r'\b\w+\b', page_text)

In [13]:
# Charger le modèle spaCy pour le français
nlp = spacy.load("fr_core_news_sm")

def racine (text):
    # Traiter le document avec spaCy
    doc = nlp(text)

    # Lemmatiser chaque mot
    lemmatized_words = [token.lemma_ if token.pos_ != "PROPN" else token.text for token in doc]

    # Joindre les mots lemmatisés pour reconstituer le document
    lemmatized_document = " ".join(lemmatized_words)

    return lemmatized_document

In [14]:
#racine , split et retourne seulement les mots de la requete qui figure dans le dictionnaire
def check_stem_and_split (query):
    stemmed_splitted = text_to_words (racine(query))
    return [word.lower() for word in stemmed_splitted if word in d  ]

In [15]:
#retourne les pages dont l'id est dans pages_id
def get_pages_by_ids(pages_id):
    return [page for page in pages_list if page[1] in pages_id]

Traitement des pages de la requetes:

In [16]:
#retourne un dictionnaire des mots de la requete avec leurs pages à partir de la structure mot-pages
def find_pages_for_query(query_splitted_rooted):
    
    pages_for_query = {}

    #Iterate through each word in the query
    for word in query_splitted_rooted:
        #Check if the word exists in the word dictionary
        if word in d:
            #Retrieve the list of tuples
            pages_tf_dict = d[word][1]

            #Extract the page IDs for the word
            pages_for_word = [page_id for page_id in pages_tf_dict.keys()]

            #Store the list of page IDs for the word
            pages_for_query[word] = pages_for_word
        else:
            # If the word is not found in the dictionary, set an empty list for its pages
            pages_for_query[word] = []

    return pages_for_query

In [17]:
#algortihme d'intesection des pages (algo basic)
def find_intersection_of_pages(pages_for_query, query_words):
    
    #Initialize a set with the pages containing the first query word
    intersection_pages = set(pages_for_query.get(query_words[0], []))

    #Iterate through each subsequent word in the query
    for word in query_words[1:]:
        
        #Get the set of pages containing the current query word
        word_pages = set(pages_for_query.get(word, []))

        #Take the intersection with the current intersection_pages
        intersection_pages = intersection_pages.intersection(word_pages)

    return list(intersection_pages)

In [18]:
#trouve les pages pour chaque mots de la requete et retourne les informations des pages de l'intersection
def find_all_query_pages( mots_racine_requete ) :
    query_pages = find_pages_for_query(mots_racine_requete) #trouver les pages pour chaque mot de la requete
    #print (query_pages)
    all_containing_pages = find_intersection_of_pages(query_pages,mots_racine_requete) #les pages contenant tout les mots de la requete
    #print (all_containing_pages)
    return get_pages_by_ids(all_containing_pages)

In [19]:
#Calcul le score de Frequece d'une page
def calculate_Frequancy_score_page_word (page_id,request_words):
    F_score = 0
    for word in request_words:
        idf = d[word][0]
        pages_Tf_dict = d[word][1]
        F_score += idf * pages_Tf_dict[page_id]
    return F_score

In [20]:
#filtre pour les pages contenant certains mots
def check_banned_words (title,words = ["bot","wikipédia:","projet:"]) :
    for words in words :
        if words in title.lower():
            return False
    return True


VERSION 0 : basic du cours

In [21]:
def handle_query(query):
    scores = {} #pour sauvegarder le score des pages 
    mots_racine_requete = check_stem_and_split(query)
    if not mots_racine_requete: return scores #si aucun mot n'est dans le dictionnaire , renvoie le dict vide
    intersection = find_all_query_pages(mots_racine_requete)
    for title, id, _ in intersection: #pour chaque page de l'intersection
        
        if not check_banned_words(title): #appliquer le filtre sur le titre
            continue


        #F(d,r)
        freq_score = calculate_Frequancy_score_page_word(id, mots_racine_requete)
        

        #P(d)
        page_rank_score = pages_rank[id]

        
        # s = F(d,r) + P(d)
        final_score = freq_score +  page_rank_score  
        
        #sauvegarder le score
        scores[title] = (final_score,freq_score,page_rank_score)

    # Sort the pages by their final scores in descending order
    sorted_scores = sorted(scores.items(), key=lambda x: x[1][0], reverse=True)
    
    return sorted_scores

In [22]:
#Example d'usage
query = "android"
relevant_pages = handle_query(query)

# Print the top 5 relevant pages
for i, (title, score) in enumerate(relevant_pages[:10], 1):  # Limit to the first 5 elements
    print(f"{i}. {title}: {score}")

1. Grove Street Games: (0.6323245949132473, 0.6323131785465744, 1.1416366672890917e-05)
2. The Game Bakers: (0.6009145640399003, 0.6009031372767627, 1.142676313763581e-05)
3. TouchWiz: (0.6007244832123283, 0.6007130653604461, 1.1417851882140187e-05)
4. Android SDK: (0.5996117440411275, 0.5996003291596639, 1.1414881463641647e-05)
5. Google Podcasts: (0.5416615700490894, 0.5416501477415796, 1.1422307509888e-05)
6. Samsung Health: (0.5311391666471563, 0.5311277383988094, 1.1428248346885079e-05)
7. Google TV: (0.505800458650005, 0.5057890229756119, 1.1435674393131432e-05)
8. Android One: (0.503223483612999, 0.5032120687315353, 1.1414881463641647e-05)
9. Nexus 7 (2013): (0.49723343955618854, 0.4972220217043064, 1.1417851882140187e-05)
10. Kindle Fire: (0.48277393305931016, 0.4827624988701263, 1.1434189183882162e-05)


TESTING: VERSION 1

In [23]:
def handle_query(query):
    scores = {}
    mots_racine_requete = check_stem_and_split(query)
    if not mots_racine_requete: return scores #si aucun mot n'est dans le dictionnaire , renvoie le dict vide
    intersection = find_all_query_pages(mots_racine_requete)
    for title, id, _ in intersection:
        if not check_banned_words(title):
            continue
        
        #score de freq
        freq_score = calculate_Frequancy_score_page_word(id, mots_racine_requete)
        
        #augmenter la valeur du page rank
        page_rank_score = 100*pages_rank[id]
        if page_rank_score < 0.001 : #si pagerank trop petit
            final_score = page_rank_score #le score final est le score du pagerank
        elif page_rank_score > freq_score : #sinon si le page_rank est plus grand que le score de frequance
            final_score = freq_score #le score final est le score de frequance
        else :
            final_score = freq_score + page_rank_score #sinon formule du cour
        
        
        scores[title] = (final_score, freq_score, page_rank_score)

    # Tri par ordre decroissant
    sorted_scores = sorted(scores.items(), key=lambda x: x[1][0], reverse=True)
    
    return sorted_scores

In [24]:
#dict mot-pages sans normalisation
dict_path = 'mot_pages_100k_without_norm'

with open(dict_path, 'rb') as pickle_file:
    d=pickle.load(pickle_file)


In [25]:
#page rank normalisé 
pages_rank_path = 'nomalizerd_pages_rank'

with open(pages_rank_path, 'rb') as pickle_file:
    pages_rank=pickle.load(pickle_file)

In [26]:
# Example usage
query = "numpy"
relevant_pages = handle_query(query)

#top 5 relevant pages
for i, (title, score) in enumerate(relevant_pages[:10], 1):  # Limit to the first 5 elements
    print(f"{i}. {title}: {score}")

1. NumPy: (21.33134715037841, 21.21748435504678, 0.11386279533163148)
2. Fortran: (14.57716477902176, 11.887156239312036, 2.6900085397097246)
3. Tas (informatique): (11.929854787561402, 11.887156239312036, 0.04269854824936586)
4. Matrice vide: (11.901389088728486, 11.887156239312036, 0.014232849416449873)
5. SageMath: (9.435617106854663, 9.136727269109134, 0.2988898377455285)
6. Liste de logiciels libres en géologie: (9.165192967942033, 9.136727269109134, 0.028465698832899747)


In [27]:
# Example usage
query = "google pixel"
relevant_pages = handle_query(query)

#top 5 relevant pages
for i, (title, score) in enumerate(relevant_pages[:20], 1):  # Limit to the first 5 elements
    print(f"{i}. {title}: {score}")

1. Android: (36.42777089593001, 21.952963039396813, 14.474807856533193)
2. Google: (31.566710318288003, 17.760846384328115, 13.805863933959886)
3. Google Pixel: (22.76813663906744, 22.696972391985156, 0.07116424708228183)
4. Pixel art: (20.738135712080098, 20.68120431441428, 0.05693139766581574)
5. Chromebook Pixel: (18.77371576690233, 18.731017218652966, 0.04269854824936586)
6. Assistant Google: (17.207369965209853, 17.121972868711122, 0.08539709649873171)
7. Liste des acquisitions de Google: (16.9812558369038, 16.938557288654433, 0.04269854824936586)
8. Google Nexus: (16.694305803829803, 16.637374406163985, 0.05693139766581574)
9. Chromebook: (16.096867434097287, 15.826443295184674, 0.2704241389126125)
10. Google Lens: (15.769670261320028, 15.755437411903578, 0.014232849416449873)
11. JavaScript: (15.646359216947477, 8.003319080311801, 7.643040136635677)
12. Google Street View: (15.128808211343642, 14.801452674765198, 0.3273555365784445)
13. YouTube: (14.312795181893769, 14.312795181

VERSION 2 (BEST SO FAR):

In [28]:
#dictionnaire mot_pages de 100_000 mots sans normalisation

dict_path = 'mot_pages_100k_without_norm'

with open(dict_path, 'rb') as pickle_file:
    d=pickle.load(pickle_file)

In [29]:
#page rank non normalisé

pages_rank_path = 'pages_rank'

with open(pages_rank_path, 'rb') as pickle_file:
    pages_rank=pickle.load(pickle_file)

In [30]:
def handle_query(query):
    scores = {} #le score des pages 
    mots_racine_requete = check_stem_and_split(query)
    if not mots_racine_requete: return scores #si aucun mot n'est dans le dictionnaire , renvoie le dict vide
    intersection = find_all_query_pages(mots_racine_requete)
    for title, id, _ in intersection: #pour chaque page de l'intersection
        if not check_banned_words(title): #appliquer le filtre sur le titre
            continue
        
        #F(d,r)
        freq_score = calculate_Frequancy_score_page_word(id, mots_racine_requete)
        
        #P(d)
        page_rank_score = pages_rank[id]
        
        # s = F(d,r) * P(d)
        final_score = freq_score *  page_rank_score  #best so far
        
       
        scores[title] = (final_score,freq_score,page_rank_score)

    #tri par ordre decroissant
    sorted_scores = sorted(scores.items(), key=lambda x: x[1][0], reverse=True)
    
    return sorted_scores

In [31]:
# Example usage
query = "google pixel"
relevant_pages = handle_query(query)

#top 5 relevant pages
for i, (title, score) in enumerate(relevant_pages[:5], 1):  # Limit to the first 5 elements
    print(f"{i}. {title}: {score}")

1. Android: (0.00028374949526925575, 21.952963039396813, 1.2925339270149483e-05)
2. Google Pixel: (0.00025925179820469267, 22.696972391985156, 1.1422307509888e-05)
3. Pixel art: (0.0002361963594381285, 20.68120431441428, 1.1420822300638728e-05)
4. Google: (0.00022832517229617966, 17.760846384328115, 1.2855534435433782e-05)
5. Chromebook Pixel: (0.00021389579968441843, 18.731017218652966, 1.1419337091389458e-05)


In [32]:
# Example usage
query = "bitcoin"
relevant_pages = handle_query(query)

#top 5 relevant pages
for i, (title, score) in enumerate(relevant_pages[:5], 1):  # Limit to the first 5 elements
    print(f"{i}. {title}: {score}")

1. Bitcoin: (0.0002693919359489478, 23.267059502759587, 1.1578254481061374e-05)
2. Économie du bitcoin: (0.0002393062153060946, 20.956226564722314, 1.1419337091389458e-05)
3. Réseau bitcoin: (0.00020798225352854587, 18.217902375404297, 1.1416366672890917e-05)
4. Cryptomonnaie: (0.00019941517089918555, 17.449320739826135, 1.1428248346885079e-05)
5. Blockchain: (0.00018099523851773883, 15.823136525204122, 1.1438644811629972e-05)


Serveur-web :

In [33]:
from flask import Flask, request, render_template, jsonify
import random
app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/query', methods=['POST'])

def query():
    query = request.json['query']
    relevant_pages_list = handle_query(query)
    relevant_pages_dict = {title: score for title, score in relevant_pages_list}
    relevant_pages = [{'title': title, 'score': score} for title, score in relevant_pages_dict.items()]
    return jsonify(relevant_pages)

if __name__ == '__main__':

    port = 5000 + random.randint(0, 999)
    print(port)
    url = "http://127.0.0.1:{0}".format(port)
    print(url)
    app.run(use_reloader=False, debug=True, port=port)

5810
http://127.0.0.1:5810
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5810
Press CTRL+C to quit
127.0.0.1 - - [19/Mar/2024 23:39:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Mar/2024 23:39:59] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [19/Mar/2024 23:40:04] "POST /query HTTP/1.1" 200 -
